In [ ]:

rt=risk   
rt=rt[order(rt$riskScore),]                                            

pdf(file="./risk/TCGA1.pdf",width = 5,height = 5)
riskClass=rt[,"risk"]
lowLength=length(riskClass[riskClass=="low"])
highLength=length(riskClass[riskClass=="high"])
line=rt[,"riskScore"]
line[line>10]=10
plot(line, type="p", pch=20,
     xlab="Patients (increasing risk socre)", ylab="Risk score",
      col=c(rep("#2fa1dd",lowLength),rep("#f87669",highLength)) )
  abline(h=median(rt$riskScore),v=lowLength,lty=2)
  legend("topleft", c("High risk", "low Risk"),bty="n",pch=19,col=c("#f87669","#2fa1dd"),cex=1.2)
#绘制生存图
color=as.vector(rt$Status)
color[color==1]="#f87669"
color[color==0]="#2fa1dd"
  plot(rt$OS_time, pch=19,
       xlab="Patients (increasing risk socre)", ylab="Survival time (years)",
       col=color)
  legend("topleft", c("Dead", "Alive"),bty="n",pch=19,col=c("#f87669","#2fa1dd"),cex=1.2)
  abline(v=lowLength,lty=2)
dev.off()

#绘制风险热图
rt1=rt[genes]
rt1=log2(rt1+1)
rt1=t(rt1)

annotation=data.frame(type=rt[,ncol(rt)-1])
rownames(annotation)=rownames(rt)
ann_colors = list(
  type = c(high="#f87669",low="#2fa1dd"))
library(RColorBrewer)
pheatmap(rt1, 
           annotation=annotation,
            annotation_colors  = ann_colors,
           cluster_cols = FALSE,
             cluster_rows = F,
           fontsize_row=11,
           show_colnames = F,
           fontsize_col=3,
           scale = "row",cell_height = .2,
           color = colorRampPalette(c("#2fa1dd", "white", "#f87669"))(100) ,filename  = "./risk/TCGA.pdf",width =5,height = 3)

pd_exp = train_pd_exp#train_pd_exp_;tset_pd_exp_;train_pd_exp;tset_pd_exp;entire;external
riskScore=predict(cox_gene,type="risk",newdata=pd_exp)
risk=as.vector(ifelse(riskScore>median(riskScore),"high","low"))
risk = data.frame(pd_exp,riskScore,risk)
risk$Age = as.numeric(risk$Age);risk$Clinical_stage = as.factor(risk$Clinical_stage);
risk$Gender = as.factor(risk$Gender)
#risk$risk = as.factor(risk$risk)
#构建多因素COX
multiCox <- coxph(Surv(OS_time, Status) ~  Age + Gender +  Clinical_stage + 
                          riskScore+risk, data = risk)
#可以输入??datadist查看详细说明
dd=datadist(risk)
options(datadist="dd")

## 构建COX比例风险模型
f <- psm(Surv(OS_time, Status) ~ Age + Gender +  Clinical_stage + riskScore,
          data = risk,dist='lognormal')
surv <- Survival(f) # 构建生存概率函数
## time是以”天“为单位,此处绘制1年，3,5年的生存概率
nom <- nomogram(f, fun=list(function(x) surv(365, x),
                             function(x) surv(1095, x),
                             function(x) surv(1825, x) ),
                funlabel=c("1-year OS", "3-year OS", 
                           "5-year OS"))
plot(nom, xfrac=.2)
#1-year
cox1 <- cph(Surv(OS_time, Status) ~ Age + Gender + Clinical_stage +risk,
            surv=T,x=T, y=T,time.inc = 1,data=risk)
cal <- calibrate(cox1, cmethod="KM", method="boot", u=1,
                 m= 100, B=1000)
#3-year
cox3 <- cph(Surv(OS_time, Status) ~ Age + Gender + Clinical_stage +risk,
            surv=T,x=T, y=T,time.inc = 3,data=riskScore_cli2)
ca3 <- calibrate(cox3, cmethod="KM", method="boot", u=3,
                 m= 100, B=1000)
#5-year
cox5 <- cph(Surv(OS_time, Status) ~ Age + Gender + Clinical_stage +risk,
            surv=T,x=T, y=T,time.inc = 5,data=riskScore_cli2)
ca5 <- calibrate(cox5, cmethod="KM", method="boot", u=5,
                 m= 100, B=1000)
plot(cal,lwd=2,lty=1,errbar.col="black",
     xlim = c(0.5,1) ,
     ylim = c(0.4,1), 
     xlab ="Nomogram-Predicted Probability of 1,3,5Year Survival",
    ylab="Actual 1,3,5Year Survival",col="blue",sub=F)
plot(ca3,
     add = T ,
     lwd=2,lty=1,errbar.col="black",col="red",sub=F)
plot(ca5,
     add = T ,
     lwd=2,lty=1,errbar.col="black",col="green",sub=F)                